# Adding PdfItDown to your API server

PdfItDown can be integrated into your API server with just a few lines of code, and you can deploy your own file conversion endpoint, customized with your own logic and more!

**1: Setup**

In order for the server functionality to work, you need to have `pdfitdown[server]` installed.

Alternatively, you can install [`starlette`](https://starlette.dev) or a Starlette-compatible API framework such as FastAPI.

In this example we will be using Starlette, which comes bundled with the `server` dependency group for PdfItDown.

In [ ]:
! pip install -q "pdfitdown[server]>=2.0.0b2"

**2: Creating a Starlette app**

Creating a Starlette app is extremely easy, we just need to define functions that take a `Request` and return a `Response`.

We will proceed by defining a simple `/hello` GET route within our app.

In [3]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.requests import Request
from starlette.routing import Route


async def hello_route(request: Request):
    name = request.query_params.get("name", "World")
    return JSONResponse({"message": f"Hello, {name}!"})


routes = [Route(path="/hello", methods=["GET"], endpoint=hello_route, name="hello")]

app = Starlette(routes=routes)

Let's test the app with the `TestClient` provided by Starlette (which we will be using thoughout the tutorial):

In [4]:
from starlette.testclient import TestClient

client = TestClient(app=app)
response = client.get("/hello", params={"name": "PdfItDown"})
message = response.json().get("message")
print(message)

Hello, PdfItDown!


**3: Mounting PdfItDown**

Now that we have a Starlette app, we are able to mount a PdfItDown route that can handle filen conversions for us.

This route accepts only POST calls and takes a file object as input (as `multipart/form-data`) and returns a stream of bytes (the content of the converter PDF as output).

In [5]:
from pdfitdown.server import mount
from pdfitdown.pdfconversion import Converter

converter = Converter()
app = mount(
    app=app,
    converter=converter,
    path="/conversions/pdf",
    name="PdfItDown",
    uploaded_file_field="file",
)

One particular option of the `mount` function that deserves explanation is `uploaded_file_field`: it represents the name of the field, in the form data, under which the file of interest to us is stored.

This can be perfectly shown with HTML forms:

```html
<form>
<label for="file">
  Choose a file to upload:
</label>

<input
  type="file"
  id="file"
  name="file"
  accept="text/plain, text/markdown"
/>
</form>
```

If you use this form as input for an API call (as you can do with [htmx](https://htmx.org), e.g.), the file content will be passed under the `file` namespace in the form.

Let's now test the app with the`stream` method:

In [7]:
client = TestClient(app=app)

with open("data/books.xml", "rb") as f:
    file_content = f.read()
    file_name = "books.xml"
files = {"file": (file_name, file_content, "application/xml")}

response = client.stream(method="POST", url="/conversions/pdf", files=files)
with response as r:
    with open("data/books.pdf", "wb") as f:
        for chunk in r.iter_bytes():
            f.write(chunk)

You can also use the `post` method and collect the resulting bytes from the `content` attribute in the response object.

In [8]:
response = client.post(url="/conversions/pdf", files=files)

with open("data/books_nostream.pdf", "wb") as f:
    f.write(response.content)

And this is it! Now, you can:

- Customize the `conversion_callback` of your PdfItDown converter to bring your own logic inside the conversion API (see [this notebook](https://github.com/AstraBert/PdfItDown/tree/main/cookbooks/custom_usage.ipynb) for a tutorial on customization)
- Add more routes with PdfItDown converters (as long as they have different paths)

Enjoy!